# 3D Interpolation with Scipy

The hall probe measurement simulation software is hamstrung by the sparseness of the available field grid data.  The Mau10 and GA05 field grids are given in Cartesian coordinates, with 25 mm spacing on the X, Y, and Z axes.  The hall probe measurement device will work most naturally in cylindrical coordinates.  In order to sample regular cylindrically symmetric data from the field grids, one must either cherry pick values that coincide with the 25x25x25 mm spacing, or generate data through interpolation.

The current interpolation method employed in the Mu2E organization for doing this is a linear 3D method.  This leads to can lead to fairly large errors in regions of large magnetic gradient, which would be unsuitable for further analysis.  We will attempt to use a more robust interpolate method and compare those results with random field data (provided in a separate file for validation purposes).

In [2]:
from mu2e.dataframeprod import DataFrameMaker
from mu2e import mu2e_ext_path
import numpy as np
from scipy.interpolate import Rbf
from IPython.display import HTML


In [3]:
df = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap',use_pickle = True).data_frame

In [3]:
df_rand = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap_rand1mil',use_pickle = True).data_frame

In [8]:
#df_rand_subset = df_rand.query('-800<=X<=800 and -800<=Y<=800 and 4200<=Z<=13000')
df_rand_subset = df_rand.query('-10<=X<=10 and -10<=Y<=10 and 8200<=Z<=11000')

We have our dataframes queued up.  We will attempt to interpolate a regular grid from `df`, and test values from `df_rand` to determine our accuracy.  Let's try to use scipy.interpolate.Rbf in a simple case, first:

In [7]:
def cylindrical_norm(x1,x2):
    return np.sqrt(
        (x1[0,:]*np.cos(x1[1,:])-x2[0,:]*np.cos(x2[1,:]))**2 +
        (x1[0,:]*np.sin(x1[1,:])-x2[0,:]*np.sin(x2[1,:]))**2 +
        (x1[2,:]-x2[2,:])**2)

In [9]:
%run interp_studies.py
header = 'Quad&emsp;&emsp;&emsp;Lin&emsp;&emsp;&emsp;RBF'
spaces = '&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;'
out='<p>'+header+spaces+header+spaces+header+'</p>'
buffer = 100
counter = 0
print '\tBx\t\t\tBy\t\t\tBz'
samples = 100
wins_bx = [0,0,0]
wins_by = [0,0,0]
wins_bz = [0,0,0]
for i in xrange(samples):
#for i in xrange(13,14):
    my_rand = df_rand_subset.sample(1,random_state=i+10)
    df_subset = df.query('{0}<=X<={1} and {2}<=Y<={3} and {4}<=Z<={5}'.format(
        my_rand.X.values-buffer, my_rand.X.values+buffer, my_rand.Y.values-buffer, my_rand.Y.values+buffer, my_rand.Z.values-buffer, my_rand.Z.values+buffer))
    x = df_subset.X
    y = df_subset.Y
    z = df_subset.Z
    r = df_subset.R
    phi = df_subset.Phi
    bz = df_subset.Bz
    bx = df_subset.Bx
    by = df_subset.By
    b_out_true = []
    b_out_true.append(my_rand.Bx.values)
    b_out_true.append(my_rand.By.values)
    b_out_true.append(my_rand.Bz.values)
    
    b_out_rbf = []
    rbf_bx = Rbf(x, y, z, bx, function='linear', )#epsilon=9)
    rbf_by = Rbf(x, y, z, by, function='linear', )#epsilon=9)
    rbf_bz = Rbf(x, y, z, bz, function='linear', )#epsilon=9)
    b_out_rbf.append(rbf_bx(my_rand.X, my_rand.Y, my_rand.Z))
    b_out_rbf.append(rbf_by(my_rand.X, my_rand.Y, my_rand.Z))
    b_out_rbf.append(rbf_bz(my_rand.X, my_rand.Y, my_rand.Z))
    
    res_rbf = []
    res_rbf.append(abs(b_out_rbf[0]-b_out_true[0])[0])
    res_rbf.append(abs(b_out_rbf[1]-b_out_true[1])[0])
    res_rbf.append(abs(b_out_rbf[2]-b_out_true[2])[0])
    
    _, b_out_lin = interp_phi(df_subset, my_rand.X.values, my_rand.Y.values, my_rand.Z.values, plot=False)
    res_lin = []
    res_lin.append(abs(b_out_lin[0]-b_out_true[0])[0])
    res_lin.append(abs(b_out_lin[1]-b_out_true[1])[0])
    res_lin.append(abs(b_out_lin[2]-b_out_true[2])[0])
    
    _, b_out_quad = interp_phi_quad(df_subset, my_rand.X.values, my_rand.Y.values, my_rand.Z.values, plot=False)
    res_quad = []
    res_quad.append(abs(b_out_quad[0]-b_out_true[0])[0])
    res_quad.append(abs(b_out_quad[1]-b_out_true[1])[0])
    res_quad.append(abs(b_out_quad[2]-b_out_true[2])[0])
    
    
    #if (res_x>1e-4) or (res_y>1e-4) or (res_z>1e-4):
    if (res_quad[0]>1e-4 or res_quad[1]>1e-4 or res_quad[2]>1e-4):
    #    print 'bad interp:'
    #    print '\ttrue:'
    #    print '\t\t{0:.4e} {1:.4e} {2:.4e}'.format(my_rand.Bx.values[0], my_rand.By.values[0], my_rand.Bz.values[0])
    #    print '\tLaceyQuad:'
    #    print '\t\t{0:.4e} {1:.4e} {2:.4e}'.format(b_out[0][0], b_out[1][0], b_out[2][0])
    #    print '\t\t{0:.4e} {1:.4e} {2:.4e}'.format(res_lx[0], res_ly[0], res_lz[0])
    #    print '\tLacey:'
    #    print '\t\t{0:.4e} {1:.4e} {2:.4e}'.format(b_outl[0][0], b_outl[1][0], b_outl[2][0])
    #    print '\t\t{0:.4e} {1:.4e} {2:.4e}'.format(res_x[0], res_y[0], res_z[0])
    #    print

        counter +=1

    if min(res_quad[0], res_lin[0], res_rbf[0]) == res_quad[0]:
        px = '<b>{0:.2e}</b>'.format(res_quad[0]) + ' {0:.2e}'.format(res_lin[0]) + ' {0:.2e}'.format(res_rbf[0])
        wins_bx[0]+=1
    elif min(res_quad[0], res_lin[0], res_rbf[0]) == res_lin[0]:
        px = '{0:.2e}'.format(res_quad[0]) + ' <b>{0:.2e}</b>'.format(res_lin[0]) + ' {0:.2e}'.format(res_rbf[0])
        wins_bx[1]+=1
    else:
        px = '{0:.2e}'.format(res_quad[0]) + ' {0:.2e}'.format(res_lin[0]) + ' <b>{0:.2e}</b>'.format(res_rbf[0])
        wins_bx[2]+=1
        
    if min(res_quad[1], res_lin[1], res_rbf[1]) == res_quad[1]:
        py = '<b>{0:.2e}</b>'.format(res_quad[1]) + ' {0:.2e}'.format(res_lin[1]) + ' {0:.2e}'.format(res_rbf[1])
        wins_by[0]+=1
    elif min(res_quad[1], res_lin[1], res_rbf[1]) == res_lin[1]:
        py = '{0:.2e}'.format(res_quad[1]) + ' <b>{0:.2e}</b>'.format(res_lin[1]) + ' {0:.2e}'.format(res_rbf[1])
        wins_by[1]+=1
    else:
        py = '{0:.2e}'.format(res_quad[1]) + ' {0:.2e}'.format(res_lin[1]) + ' <b>{0:.2e}</b>'.format(res_rbf[1])
        wins_by[2]+=1
        
    if min(res_quad[2], res_lin[2], res_rbf[2]) == res_quad[2]:
        pz = '<b>{0:.2e}</b>'.format(res_quad[2]) + ' {0:.2e}'.format(res_lin[2]) + ' {0:.2e}'.format(res_rbf[2])
        wins_bz[0]+=1
    elif min(res_quad[2], res_lin[2], res_rbf[2]) == res_lin[2]:
        pz = '{0:.2e}'.format(res_quad[2]) + ' <b>{0:.2e}</b>'.format(res_lin[2]) + ' {0:.2e}'.format(res_rbf[2])
        wins_bz[1]+=1
    else:
        pz = '{0:.2e}'.format(res_quad[2]) + ' {0:.2e}'.format(res_lin[2]) + ' <b>{0:.2e}</b>'.format(res_rbf[2])
        wins_bz[2]+=1
        
    out += '<p>'+px+'&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;'+py+'&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;'+pz+'</p>'
wins_bx = [float(i)/samples for i in wins_bx]
wins_by = [float(i)/samples for i in wins_by]
wins_bz = [float(i)/samples for i in wins_bz]
out += '<p></p>'
out += '<p>{0:.2f}&emsp;{1:.2f}&emsp;{2:.2f}'.format(*wins_bx)
out += spaces+'&emsp;&emsp;&emsp;&emsp;'
out += '{0:.2f}&emsp;{1:.2f}&emsp;{2:.2f}'.format(*wins_by)
out += spaces+'&emsp;&emsp;&emsp;&emsp;'
out += '{0:.2f}&emsp;{1:.2f}&emsp;{2:.2f}</p>'.format(*wins_bz)

print 'bads:', counter
HTML(out)

	Bx			By			Bz
bads: 0


In [ ]:
print np.asarray(res_lin)-np.asarray(res_quin)

In [ ]:
rbf_br = Rbf(r, phi, z, br,function='gaussian', norm=cylindrical_norm)
i_br = rbf_br(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_br = df_rand_subset.Br
res =abs(i_br-t_br)
max(res)

In [ ]:
rbf_bphi = Rbf(r, phi, z, bphi,function='quintic', norm=cylindrical_norm)
i_bphi = rbf_bphi(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_bphi = df_rand_subset.Bphi
res =abs(i_bphi-t_bphi)
max(res)

In [45]:
px

In [16]:
%run interp_studies.py
#df_trimmed = interp_phi(df,25,25,9721)
#df_trimmed = interp_phi(df, -59.6429, 46.756758, 10297.7840, df_alt=df_rand_subset)
# df_trimmed, b_out = interp_phi(df, 50, 76, 10121)
df_trimmed, bs = interp_phi_cubic(df,12.5,12.5,9721-12.5, False)

In [14]:
df_trimmed

X     Y       Z        Bx        By        Bz
0  -25.0 -25.0  9671.0 -0.000635 -0.000116  1.018755
1  -25.0 -25.0  9696.0 -0.000631 -0.000115  1.018527
2  -25.0 -25.0  9721.0 -0.000627 -0.000113  1.018301
3  -25.0 -25.0  9746.0 -0.000624 -0.000112  1.018077
4  -25.0   0.0  9671.0 -0.000635  0.000000  1.018756
5  -25.0   0.0  9696.0 -0.000631  0.000000  1.018527
6  -25.0   0.0  9721.0 -0.000627  0.000000  1.018301
7  -25.0   0.0  9746.0 -0.000624  0.000000  1.018078
8  -25.0  25.0  9671.0 -0.000635  0.000116  1.018755
9  -25.0  25.0  9696.0 -0.000631  0.000115  1.018527
10 -25.0  25.0  9721.0 -0.000627  0.000113  1.018301
11 -25.0  25.0  9746.0 -0.000624  0.000112  1.018077
12 -25.0  50.0  9671.0 -0.000635  0.000232  1.018753
13 -25.0  50.0  9696.0 -0.000631  0.000229  1.018525
14 -25.0  50.0  9721.0 -0.000627  0.000227  1.018299
15 -25.0  50.0  9746.0 -0.000624  0.000224  1.018075
16   0.0 -25.0  9671.0 -0.000521 -0.000116  1.018758
17   0.0 -25.0  9696.0 -0.000518 -0.000115  1.018530
18   0.0 -25.0  9721.0 -0.000516 -0.000113  1.018304
19   0.0 -25.0  9746.0 -0.000513 -0.000112  1.018080
20   0.0   0.0  9671.0 -0.000521  0.000000  1.018759
21   0.0   0.0  9696.0 -0.000518  0.000000  1.018530
22   0.0   0.0  9721.0 -0.000516  0.000000  1.018304
23   0.0   0.0  9746.0 -0.000513  0.000000  1.018081
24   0.0  25.0  9671.0 -0.000521  0.000116  1.018758
25   0.0  25.0  9696.0 -0.000518  0.000115  1.018530
26   0.0  25.0  9721.0 -0.000516  0.000113  1.018304
27   0.0  25.0  9746.0 -0.000513  0.000112  1.018080
28   0.0  50.0  9671.0 -0.000521  0.000232  1.018756
29   0.0  50.0  9696.0 -0.000518  0.000229  1.018528
..   ...   ...     ...       ...       ...       ...
34  25.0 -25.0  9721.0 -0.000404 -0.000113  1.018306
35  25.0 -25.0  9746.0 -0.000403 -0.000112  1.018082
36  25.0   0.0  9671.0 -0.000407  0.000000  1.018761
37  25.0   0.0  9696.0 -0.000406  0.000000  1.018532
38  25.0   0.0  9721.0 -0.000404  0.000000  1.018306
39  25.0   0.0  9746.0 -0.000403  0.000000  1.018083
40  25.0  25.0  9671.0 -0.000407  0.000116  1.018760
41  25.0  25.0  9696.0 -0.000406  0.000115  1.018532
42  25.0  25.0  9721.0 -0.000404  0.000113  1.018306
43  25.0  25.0  9746.0 -0.000403  0.000112  1.018082
44  25.0  50.0  9671.0 -0.000407  0.000232  1.018758
45  25.0  50.0  9696.0 -0.000406  0.000229  1.018530
46  25.0  50.0  9721.0 -0.000404  0.000227  1.018304
47  25.0  50.0  9746.0 -0.000403  0.000224  1.018080
48  50.0 -25.0  9671.0 -0.000293 -0.000116  1.018761
49  50.0 -25.0  9696.0 -0.000293 -0.000115  1.018532
50  50.0 -25.0  9721.0 -0.000293 -0.000113  1.018306
51  50.0 -25.0  9746.0 -0.000293 -0.000112  1.018083
52  50.0   0.0  9671.0 -0.000293  0.000000  1.018762
53  50.0   0.0  9696.0 -0.000293  0.000000  1.018533
54  50.0   0.0  9721.0 -0.000293  0.000000  1.018307
55  50.0   0.0  9746.0 -0.000293  0.000000  1.018083
56  50.0  25.0  9671.0 -0.000293  0.000116  1.018761
57  50.0  25.0  9696.0 -0.000293  0.000115  1.018532
58  50.0  25.0  9721.0 -0.000293  0.000113  1.018306
59  50.0  25.0  9746.0 -0.000293  0.000112  1.018083
60  50.0  50.0  9671.0 -0.000293  0.000232  1.018759
61  50.0  50.0  9696.0 -0.000293  0.000229  1.018531
62  50.0  50.0  9721.0 -0.000293  0.000227  1.018304
63  50.0  50.0  9746.0 -0.000293  0.000224  1.018081

[64 rows x 6 columns]

In [19]:
(df_trimmed.Bx[13]-bs[0])/df_trimmed.Bx[13]

1.6070889931830089e-05